In [1]:
import random
import numpy as np
import numba
from skimage.measure import label
from skimage import morphology
from skimage.color import label2rgb
import matplotlib.pyplot as plt
import h5py

In [2]:
@numba.jit(nopython=True, nogil=True)
def get_boudary_from_label(labeled, neighbors=4):
    """
    Get boudary map from labeld image
    Each pixel have its own label. If one pixel find different label in its neighborhood, 
    the pixel should be boudary pixel and set to 255. Otherwise, it should be set to 0.
    :param labeled: ndarray
        The labeled image.
    :param neighbors: {4, 8}, int, optional
        Default 4 neighbors for pixel neighborhood.
    :return: edges, ndarray
        The edge map for labeled.
    """
    boudary = np.zeros(labeled.shape, dtype=np.uint8)
    for row in range(0, labeled.shape[0]):
        for col in range(0, labeled.shape[1]):
            value = labeled[row, col]
            is_boudary = False
            if value == 0:
                boudary[row, col] = 255
                continue
            if neighbors == 4:
                if row != 0 and value != labeled[row-1, col]:
                    is_boudary = True
                elif col != 0 and value != labeled[row, col-1]:
                    is_boudary = True
                elif row != labeled.shape[0]-1 and value != labeled[row+1, col]:
                    is_boudary = True
                elif col != labeled.shape[1]-1 and value != labeled[row, col+1]:
                    is_boudary = True
            elif neighbors == 8:
                if row != 0 and value != labeled[row-1, col]:
                    is_boudary = True
                elif row != 0 and col != 0 and value != labeled[row-1, col-1]:
                    is_boudary = True
                elif row != 0 and col != labeled.shape[1]-1 and value != labeled[row-1, col+1]:
                    is_boudary = True
                elif row != labeled.shape[0]-1 and value != labeled[row+1, col]:
                    is_boudary = True
                elif row != labeled.shape[0]-1 and col != 0 and value != labeled[row+1, col-1]:
                    is_boudary = True
                elif row != labeled.shape[0]-1 and col != labeled.shape[1]-1 and value != labeled[row+1, col+1]:
                    is_boudary = True
                elif col != 0 and value != labeled[row, col-1]:
                    is_boudary = True
                elif col != labeled.shape[1]-1 and value != labeled[row, col+1]:
                    is_boudary = True
            if is_boudary:
                boudary[row, col] = 255
    return boudary

In [3]:
# visualization of virtual image data
file_h5 = h5py.File("data_volumes.hdf5", "r")

virtual_group = file_h5["virtual"]
h, w, d = virtual_group["shape"][0], virtual_group["shape"][1],  virtual_group["shape"][2]
select_index = random.randint(0, d)

label_image = virtual_group["label"][:, :, select_index]
label_color_image = label2rgb(label_image)

boundary_image = virtual_group["boundary"][:, :, select_index]
# or use the boudary function:
boundary_image = get_boudary_from_label(label_image)
boundary_image = morphology.skeletonize(boundary_image / 255) * 255  # get single pixel width

plt.figure(figsize=(30,30))
plt.subplot(121)
plt.imshow(label_color_image)
plt.axis('off')
plt.title("label", fontsize=30)
plt.subplot(122)
plt.imshow(boundary_image, cmap="gray")
plt.axis('off')
plt.title("boundary", fontsize=30)
plt.show()

NameError: name 'edges' is not defined

In [ ]:
# visualization of real image data
file_h5 = h5py.File("data_volumes.hdf5", "r")

virtual_group = file_h5["real"]
h, w, d = virtual_group["shape"][0], virtual_group["shape"][1],  virtual_group["shape"][2]
select_index = random.randint(0, d)

origin_image = virtual_group["image"][:, :, select_index]

label_image = virtual_group["label"][:, :, select_index]
label_color_image = label2rgb(label_image)

boundary_image = virtual_group["boundary"][:, :, select_index]
# or use the boudary function:
boundary_image = get_boudary_from_label(label_image)
boundary_image = morphology.skeletonize(boundary_image / 255) * 255  # get single pixel width

show_length = 200
plt.figure(figsize=(30,30))
plt.subplot(131)
plt.imshow(origin_image[0:show_length, 0:show_length], cmap="gray")
plt.axis('off')
plt.title("image", fontsize=30)
plt.subplot(132)
plt.imshow(label_color_image[0:show_length, 0:show_length])
plt.axis('off')
plt.title("label", fontsize=30)
plt.subplot(133)
plt.imshow(boundary_image[0:show_length, 0:show_length], cmap="gray")
plt.axis('off')
plt.title("boundary", fontsize=30)
plt.show()